In [1]:
import os
import pandas as pd
import xml.etree.ElementTree as ET

xml_folder = r'C:\Users\ayo\Downloads\internship_dataset\List_of Beneficiary _of_EU_FUNDS'

for file in os.listdir(xml_folder):
    if file.endswith('.xml'):
        xml_path = os.path.join(xml_folder, file)
        tree = ET.parse(xml_path)
        root = tree.getroot()
        rows = []

        # Iterate over each <zadatel>
        for zadatel in root.findall('.//zadatel'):
            name = zadatel.findtext('jmeno_nazev')
            obec = zadatel.findtext('obec')
            okres = zadatel.findtext('okres')

            # Standard payments
            for platba in zadatel.findall('./platby/platba'):
                fund = platba.findtext('fond_typ_podpory')
                measure = platba.findtext('opatreni')
                cr_fund = platba.findtext('zdroje_cr')
                eu_fund = platba.findtext('zdroje_eu')
                total = platba.findtext('celkem_czk')
                rows.append({
                    'Jméno / Název': name,
                    'Obec': obec,
                    'Okres': okres,
                    'Fond': fund,
                    'Opatření': measure,
                    'Zdroje ČR': float(cr_fund) if cr_fund else None,
                    'Zdroje EU': float(eu_fund) if eu_fund else None,
                    'Celkem (Kč)': float(total) if total else None,
                    'Typ platby': 'Standard'
                })

            # PVP (národní) payments
            for platba_pvp in zadatel.findall('./platby_pvp/platba_pvp'):
                fund = platba_pvp.findtext('fond_typ_podpory')
                measure = platba_pvp.findtext('opatreni')
                total = platba_pvp.findtext('celkem_czk')
                rows.append({
                    'Jméno / Název': name,
                    'Obec': obec,
                    'Okres': okres,
                    'Fond': fund,
                    'Opatření': measure,
                    'Zdroje ČR': None,
                    'Zdroje EU': None,
                    'Celkem (Kč)': float(total) if total else None,
                    'Typ platby': 'PVP (národní)'
                })

        if rows:
            df = pd.DataFrame(rows)
            csv_path = os.path.splitext(xml_path)[0] + '.csv'
            df.to_csv(csv_path, index=False)